In [237]:
from bs4 import BeautifulSoup
import requests
import fitz
import pdfplumber
import hashlib
import spacy
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
#import DetectionAcessToEducation.trainingaccessdetection as tad
import pandas as pd
import pickle
import spacy
import time
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [238]:
# from vehelper import *

In [239]:
import sqlalchemy
from sqlalchemy.sql import table, column, select, update, insert
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy.orm import Session
import datetime

In [257]:
# Datenbank
dbEngine=sqlalchemy.create_engine('sqlite:///weiterbildung.sqlite') # ensure this is the correct path for the sqlite file. 
conn = dbEngine.connect()
session = Session(dbEngine)

In [241]:
# Browser
chrome_options = webdriver.FirefoxOptions()#ChromeOptions()
chrome_options.headless = True
driver = webdriver.Firefox(options=chrome_options)
driver.implicitly_wait(15)

/var/folders/x7/5php5bs9399c34w93tdfd7680000gp/T/ipykernel_70541/431154803.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  chrome_options.headless = True


In [258]:
insp = inspect(dbEngine)
print(insp.get_table_names())

['weiterbildung', 'weiterbildung-kategorien', 'weiterbildungen-it-numbers']


In [259]:
df_it = pd.read_csv("it-fortbildungen-large.csv") #, sep='|', header=None,                     names=["ID", "Title", "Search"])

In [260]:
df_numbers = pd.DataFrame()

In [261]:
# Absolute Zahlen
session = Session(dbEngine)
for index, row in df_it.iterrows():
    print ("Row "+str(index)+"/"+str(len(df_it))+"           ", end="\r")
    url = "https://web.arbeitsagentur.de/weiterbildungssuche/suche?sty=2&seite=0&bildungsziel="+str(row['Search']) # 1 = Aufstieg, 2=Berufliche Qualifizierung
    driver.get(url)
    time.sleep (8)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    a1 = soup.find("span", {"class": "wbsuche-lib-ergebniszaehler"})
    result = 0
    if a1 != None:
        result = int(a1.get_text(" ").replace(".",""))
    df_numbers.at[pd.to_datetime('today').normalize(), row['Search']] = result
    resultn = result
    stmt = text("INSERT INTO 'weiterbildungen-it-numbers' (date, category, result) VALUES ('"+str(datetime.datetime.now())+"', "+str(row['Search'])+", "+str(result)+")  ")
    session.execute(stmt)
    session.commit()
#    print (stmt)
#with dbEngine.connect() as conn:
#    ret = conn.execute(stmt)
#print (ret)
    #sqliteConnection.commit()


In [262]:
df_numbers

,128456,122784,122786,122787,122788,122790,122791,122792,122793,122795,...,122837,124142,122839,122840,122841,122842,122843,122846,122847,124145
2023-06-04,14016.0,540.0,5.0,10.0,4.0,44710.0,2636.0,16498.0,8316.0,0.0,...,0.0,4.0,0.0,0.0,4.0,12.0,0.0,37943.0,5374.0,11391.0


In [263]:
df_numbers.to_csv("it-fortbildungen-large-numbers.csv")

In [265]:
df_stat = pd.DataFrame()
maxpages = 3
lurl = "https://web.arbeitsagentur.de/weiterbildungssuche"
for index, row in df_it.iterrows():
    print ("\t\t\t\t\t\t                   Row "+str(index)+"/"+str(len(df_it))+" ("+str(row['Title'])+")                        ", end="\r")
    if df_numbers[row['Search']][0] > 0:   
        linkliste = []
        #url = "https://web.arbeitsagentur.de/weiterbildungssuche/suche?sty=2&seite=0&bildungsziel="+str(row['Search'])
        for i in range(0,maxpages):
            url = lurl+"/suche?sty=2&seite="+str(i)+"&bildungsziel="+str(row['Search']) # 1 = Aufstieg, 2=Berufliche Qualifizierung
            print ("Handling page "+str(i)  , end="\r")
            driver.get(url)
            time.sleep (8)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            for link in soup.find_all('a'):
                href = link.get('href')
                if href == None:
                    continue
                #print (str(link)+ str("angebot" in href))
                if "angebot" in href:
                    linkliste.append ("https://web.arbeitsagentur.de/"+href)
        linkliste = list(set(linkliste))
        print ("Found "+str(len(linkliste)))#  , end="\r")
        
        # Nun die einzelnen Seiten bearbeiten...
        df = pd.DataFrame()
        i = 0
        for j  in range(0,len(linkliste)): # len(linkliste)
            try:
                url2 = linkliste[j].replace("r.dea","r.de/weiterbildungssuche/a")
                url2 = url2.replace("tur.de/an","tur.de/weiterbildungssuche/an")
                #print (url2)
                #chrome_options = webdriver.FirefoxOptions()#ChromeOptions()
                #chrome_options.headless = True
                #driver = webdriver.Firefox(options=chrome_options)
                #driver.implicitly_wait(5)
                driver.get(url2)
                #l = driver.find_element (By.XPATH, '//button[@class="ba-btn ba-btn-contrast"]')
                #l = driver.find_element (By.CLASS_NAME, 'ba-btn ba-btn-contrast')
                #driver.execute_script("arguments[0].click();", l);
                #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/bahf-cd-modal/div/div/div/div[3]/button[1]'))).click()
                time.sleep (7)
                html = driver.page_source
                soup2 = BeautifulSoup(html, "html.parser")
                i = i + 1
                #print ("========"+str(j))
                #print (url2)
                titel = soup2.find("h1", {"id": "detail-seite-ueberschrift"}).get_text(" ")
                a1 = soup2.find("dd", {"id": "_marg_abinfo_bildungsart"})
                if a1 != None:
                    art = a1.get_text(" ")
                a1 = soup2.find("dd", {"id": "_marg_wbinfo_bildungsart"})    
                if a1 != None:
                    art = a1.get_text(" ")
                a1 = soup2.find("dd", {"id": "_marg_abinfo_abschlussbez"})
                ab = ""
                if a1 != None:
                    ab = a1.get_text(" ")
                a1 = soup2.find("dd", {"id": "_marg_wbinfo_abschlussbez"})   
                if a1 != None:
                    ab = a1.get_text(" ")
                uform = ""
                a1 = soup2.find("dd", {"id": "_marg_abinfo_uform"})  
                if a1 != None:
                    uform = a1.get_text(" ")   
                a1 = soup2.find("dd", {"id": "_marg_wbinfo_uform"})  
                if a1 != None:
                    uform = a1.get_text(" ") 
                anbieter = ""
                a1 = soup2.find("section", {"id": "anbieter"})   
                if a1 != None:
                    anbieter = a1.get_text(" ")
                ad = anbieter
                if "Adresse" in ad:
                    ad = ad.split("Adresse")[1]
                if "Telefon" in ad:
                    ad = ad.split("Telefon")[0]
                # GeoLocation
                location = geolocator.geocode(ad, addressdetails=True)
                schulart = ""
                a1 = soup2.find("dd", {"id": "_marg_abinfo_schulart"})    
                if a1 != None:
                    schulart = a1.get_text(" ")
                a1 = soup2.find("dd", {"id": "_marg_wbinfo_schulart"})    
                if a1 != None:
                    schulart = a1.get_text(" ")
                a1 = soup2.find("section", {"id": "ausbildungsinhalte"})
                texti = "" 
                if a1 != None:
                    texti = a1.get_text(" ")
                a1 = soup2.find("section", {"id": "inhalte"}) 
                if a1 != None:
                    texti = a1.get_text(" ")   
                a1 = soup2.find("section", {"id": "detail_inhalte"})
                if a1 != None:
                    texti = a1.get_text(" ")
                if ab != None:
                    abschluss = str(ab)
                    kldb = ""
                    # getKldB(abschluss)
                else:
                    abschlus = "--"
                    kldb = "???"
                a1 = soup2.find("dd", {"id": "_marg_abinfo_zugang"})
                if a1 != None:
                    zugang = a1.get_text(" ") 
                a1 = soup2.find("dl", {"id": "detail_zuganginfo"})
                if a1 != None:
                    zugang = a1.get_text(" ")
                if zugang ==None:
                    print ("Error, ZG None")
                    continue
                # zugaenge = accessdetection(zugang)
                # skills = getSkills(texti)
                inhalt = soup2.find("p", {"id": "detail_inhalte"})
                #print (str(titel) + " ("+str(art)+")")
                #print (abschluss + "->" +kldb)
                ct = ""
                st = ""
                adr = ""
                lat = 0
                long = 0
                if location != None:
                    adr = location.address
                    lat = location.latitude
                    long = location.longitude
                    if 'town' in location.raw['address']:
                        ct = location.raw['address']['town']
                    elif 'city' in location.raw['address']:
                        ct = location.raw['address']['city']
                    if 'state' in location.raw['address']:
                        st = location.raw['address']['state']
                data = {"url" : url2,
                          "title" : titel,
                                    # "beschreibung" : texti,
                                    # "length" : len(texti.split(" "))+1,
                                # "outcome": len(skills),
                                # "outcomeskills": ";".join(skills),
                                # "uform":uform,
                        "schulart":schulart,
                        "anbieter":anbieter,
                        "adresse":adr,
                        "stadt":ct,
                        "land":st,
                        "latitude":lat, "longitude":long,
                             "art":art, "abschluss":ab, "KldB":kldb}

                iden = str(url2+"-"+hashlib.sha1(bytes(titel, 'utf-8')).hexdigest())
                check = text('SELECT EXISTS(SELECT 1 FROM "weiterbildung" WHERE id="'+iden+'");')
                r = session.execute(check).all()[0][0]
                if r == 0:
                    stmt = text("""INSERT INTO 'weiterbildung' (id, url, title, schulart, anbieter, adresse, stadt, land, latitude, longitude, art, abschluss, KldB, category, date) 
                    VALUES ('"""+iden+"""','"""+url2+"""','"""+titel+"""','"""+schulart+"""','"""+anbieter+"""',
                    '"""+str(adr)+"""','"""+ct+"""','"""+st+"""','"""+str(lat)+"""',
                    '"""+str(long)+"""','"""+art+"""','"""+ab+"""','"""+str(kldb)+"""',
                    '"""+str(row['Search'])+"""','"""+str(datetime.datetime.now())+"""')  """)
                    #print (stmt)
                    session.execute(stmt)
                    session.commit()
                stmt = text(""" INSERT  OR IGNORE INTO 'weiterbildung-kategorien' (id, category) 
                values ('"""+iden+"""','"""+str(row['Search'])+"""') """)
                session.execute(stmt)
                session.commit()
                # for z in zugaenge:
                #     data["V"+z]=1
                #     stmt = text(""" INSERT  OR IGNORE INTO 'weiterbildung-zugaenge' (id, zugang) 
                #         values ('"""+iden+"""','"""+str(z)+"""') """)
                #     session.execute(stmt)
                #     session.commit()
                # for z in skills:
                #     data["O"+z]=1
                #     stmt = text(""" INSERT  OR IGNORE INTO 'weiterbildung-ne' (id, ne) 
                #         values ('"""+iden+"""','"""+str(z)+"""') """)
                #     session.execute(stmt)
                #     session.commit()                   
                df2 = pd.DataFrame(data, index=[0])
                df = pd.concat([df, df2], ignore_index=True)
            except Exception as e: print("Exception: "+str(e))
                #print ("Fehler!")
        # Speichern etc.
        # stringCheck = pd.to_datetime('today').normalize()
        # if(str(isinstance(stringCheck, str))):
        #     print("stringCheck",stringCheck)
        #     datum = stringCheck
        # else: 
        #     print("stringCheck else",stringCheck)
        #     datum = str(stringCheck).split(" ")[0]
        #df.to_csv (str(row['Search'])+".csv")
        # dft = df.fillna(0).mean()
        # dft["id"] = row['Search']
        # dfo = pd.DataFrame()
        # df3 = pd.concat([dfo, dft]).T
        # df_stat = pd.concat([df_stat, df3])#, ignore_index=True)
        # print("df_stat",df_stat)
        
    #if index>10:
    #    break
        
        
        

Found 60 page 2          Row 0/99 (App-Entwicklung, -Programmierung)                        
Found 60 page 2          Row 1/99 (Betriebssanitätswesen, Erste Hilfe)                        
Exception: 'NoneType' object has no attribute 'get_text'
Found 5g page 2          Row 2/99 (Tierärztliche Fachfortbildung)                        
Found 10 page 2          Row 3/99 (Naturheilkunde und alternative Therapien in der Tiermedizin, Tierphysiotherapie)                        
Found 4g page 2          Row 4/99 (Tierpsychologie)                        
Found 60 page 2          Row 5/99 (Gesundheitsmanagement, Case Management)                        
Found 60 page 2          Row 6/99 (Medizinische Assistenz - allgemein)                        
Found 60 page 2          Row 7/99 (Praxismanagement)                        
Found 60 page 2          Row 8/99 (Abrechnung, Buchführung - Gesundheitswesen)                        
Found 60 page 2          Row 10/99 (Medizinische Dokumentation/Kodierung)  

In [ ]:
base_url = "https://web.arbeitsagentur.de/ausbildungssuche/berufsausbildung-suche?sty=0&seite={}&atyp=102&kat=0"
page = 0
url2 = base_url.format(page)
driver.get(url2)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
pagination_elem = soup.find("div", {"class": "paginierung"})
total_pages = int(pagination_elem.find_all("a")[-2].get_text(" "))

while page < total_pages:
    
    url = base_url.format(page)
    driver.get(url)
    time.sleep (8)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    page_trainings = soup.find_all("article", {"class": "ba-layout-tile absuche-tile"})
    
    for training in page_trainings:
        
        training_title_elem = training.find("a", {"class": "h5 ba-link-primary"})
        training_title = training_title_elem.get_text(" ") if training_title_elem else "N/A"
        href = training_title_elem["href"] if training_title_elem else ""

        start_with = "https://web.arbeitsagentur.de/"
        i = 0
        while not href.startswith(start_with[i:]):
            i += 1
        link = start_with[:i] + href
        driver.get(link)
        time.sleep (8)
        training_html = driver.page_source
        training_soup = BeautifulSoup(training_html, "html.parser")

        # Extract additional details
        start_date_elem = training_soup.find("span", {"id": "_marg_abinfo_beginn"})
        start_date = start_date_elem.get_text(" ") if start_date_elem else "Not mentioned"

        end_date_elem = training_soup.find("span", {"id": "_marg_abinfo_ende"})
        end_date = end_date_elem.get_text(" ") if end_date_elem else "N/A"

        individual_entry_elem = training_soup.find("span", {"id": "_marg_abinfo_einstieg"})
        individual_entry = individual_entry_elem.get_text(" ") if individual_entry_elem else "N/A"

        duration_elem = training_soup.find("span", {"id": "_marg_abinfo_dauer"})
        duration = duration_elem.get_text(" ") if duration_elem else "N/A"

        form_of_teaching_elem = training_soup.find("span", {"id": "_marg_abinfo_uform"})
        form_of_teaching = form_of_teaching_elem.get_text(" ") if form_of_teaching_elem else "N/A"

        degree_type_elem = training_soup.find("span", {"id": "_marg_abinfo_abschlussart"})
        degree_type = degree_type_elem.get_text(" ") if degree_type_elem else "N/A"

        degree_designation_elem = training_soup.find("span", {"id": "_marg_abinfo_abschlussbez"})
        degree_designation = degree_designation_elem.get_text(" ") if degree_designation_elem else "N/A"
   
        examining_body_elem = training_soup.find("span", {"id": "_marg_abinfo_pruefendestelle"})
        examining_body = examining_body_elem.get_text(" ") if examining_body_elem else "N/A"
   
        type_of_school_elem = training_soup.find("span", {"id": "_marg_abinfo_schulart"})
        type_of_school = type_of_school_elem.get_text(" ") if type_of_school_elem else "N/A"

        training_provider_elem = training_soup.find("div", {"id": "anbieter-name"})
        training_provider = training_provider_elem.get_text(" ") if training_provider_elem else "N/A"

        number_of_participants_elem = training_soup.find("span", {"id": "_marg_abinfo_teilnehmerzahl"})
        number_of_participants = number_of_participants_elem.get_text(" ") if number_of_participants_elem else "N/A"
        
        max_participants = 0
        parts = number_of_participants.split("-")
        if len(parts) >= 2:
          max_participants_text = parts[1].strip().split()[0]
          max_participants = max_participants_text if max_participants_text.isdigit() else 0

        place_of_training_elem = training_soup.find("div", {"id": "detail-header__ort"})
        place_of_training = place_of_training_elem.get_text(" ")[3:].strip() if place_of_training_elem else "N/A"

        state_of_training_elem = training_soup.find("div", {"id": "detail-header__land"})
        state_of_training = state_of_training_elem.get_text(" ")[4:].strip()  if state_of_training_elem else "N/A"

        funding_approval_with_edu_voucher_elem = training_soup.find("span", {"id": "_marg_abinfo_bildungsgutschein"})
        funding_approval_with_edu_voucher = funding_approval_with_edu_voucher_elem.get_text(" ") if funding_approval_with_edu_voucher_elem else "N/A"

        event_id_elem = training_soup.find("span", {"id": "_veranstaltung"})
        event_id = event_id_elem.get_text(" ") if event_id_elem else "N/A"  

        provider_id_elem = training_soup.find("span", {"id": "_anbieter_id"})
        provider_id = provider_id_elem.get_text(" ") if provider_id_elem else "N/A"  

        update_date_elem = training_soup.find("span", {"id": "_aktualisierungsdatum"})
        update_date = update_date_elem.get_text(" ")  if update_date_elem else "N/A"
        
        check = text('SELECT EXISTS(SELECT 1 FROM "schulisch_ausbildung" WHERE event_id="'+event_id+'");')
        r = session.execute(check).all()[0][0]
        if r == 0:
            stmt = text("INSERT INTO 'schulisch_ausbildung' (event_id,title,url,provider_id,update_date,start_date,end_date,individual_entry,duration,form_of_teaching,degree_type,degree_designation,examining_body,type_of_school,training_provider,number_of_participants,max_participants,place_of_training,state_of_training,funding_approval_with_edu_voucher) VALUES (:event_id,:training_title,:link,:provider_id,:update_date,:start_date,:end_date,:individual_entry,:duration,:form_of_teaching,:degree_type,:degree_designation,:examining_body,:type_of_school,:training_provider,:number_of_participants,:max_participants,:place_of_training,:state_of_training,:funding_approval_with_edu_voucher)")

            session.execute(stmt, {"event_id": event_id,"training_title": training_title,"link": link,"provider_id": provider_id,"update_date": update_date,"start_date": start_date,"end_date": end_date,"individual_entry": individual_entry,"duration": duration,"form_of_teaching": form_of_teaching,"degree_type": degree_type,"degree_designation": degree_designation,"examining_body": examining_body,"type_of_school": type_of_school,"training_provider": training_provider,"number_of_participants": number_of_participants,"max_participants": max_participants,"place_of_training": place_of_training,"state_of_training": state_of_training,"funding_approval_with_edu_voucher": funding_approval_with_edu_voucher })

            session.commit()
        
    page += 1  